In [ ]:

import imutils
import numpy as np
from cv2 import cv2
from numpy.typing import NDArray
from pytesseract import pytesseract

from src.show import show
from src.utils import save

roi = [
    [(982, 45), (1264, 102), 'Serie'],
    [(460, 221), (1218, 274), 'Nume'],
    [(463, 305), (1171, 354), 'Prenume'],
    [(843, 384), (1019, 434), 'Sex', 'MF'],
    [(468, 464), (853, 515), 'Data nașterii', '0123456789'],
    [(468, 543), (853, 595), 'Data emiterii', '0123456789'],
    [(468, 623), (853, 675), 'Data expirării', '0123456789'],
    [(557, 713), (1243, 770), 'Emisă de'],
]

img: NDArray = cv2.imread('data/fata.jpeg', cv2.IMREAD_GRAYSCALE)
height, width = img.shape
img = img[:height // 3, :width // 2]

template = cv2.imread('templates/template1.jpeg', cv2.IMREAD_GRAYSCALE)

In [ ]:


# orb = cv2.ORB_create(750)
detector = cv2.ORB_create(20000)
kpts1 = detector.detect(template, None)
kpts2 = detector.detect(img, None)

orb = cv2.xfeatures2d.BEBLID_create(1)
#percentage = 30

template_key_points, template_descriptors = orb.compute(template, kpts1)
# img_key_points = cv2.drawKeypoints(template, template_key_points, None)
img_key_points, img_descriptors = orb.compute(img, kpts2)

matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_BRUTEFORCE_HAMMING)
nn_matches = matcher.knnMatch(template_descriptors, img_descriptors, 2)
matched1 = []
matched2 = []
nn_match_ratio = 0.8  # Nearest neighbor matching ratio
for m, n in nn_matches:
    if m.distance < nn_match_ratio * n.distance:
        matched1.append(kpts1[m.queryIdx])
        matched2.append(kpts2[m.trainIdx])

inliers1 = []
inliers2 = []
good_matches = []
inlier_threshold = 2  # Distance threshold to identify inliers with homography check
src_points = np.float32([m.pt for m in matched2]).reshape(-1, 1, 2)
dst_points = np.float32([m.pt for m in matched1]).reshape(-1, 1, 2)

homography, _ = cv2.findHomography(dst_points, src_points, cv2.RANSAC, 5.0)
for i, m in enumerate(matched1):
    # Create the homogeneous point
    col = np.ones((3, 1), dtype=np.float64)
    col[0:2, 0] = m.pt
    # Project from image 1 to image 2
    col = np.dot(homography, col)
    col /= col[2, 0]
    # Calculate euclidean distance
    dist = np.sqrt(pow(col[0, 0] - matched2[i].pt[0], 2) + pow(col[1, 0] - matched2[i].pt[1], 2))
    if dist < inlier_threshold:
        good_matches.append(cv2.DMatch(len(inliers1), len(inliers2), 0))
        inliers1.append(matched1[i])
        inliers2.append(matched2[i])

# # create BFMatcher object
# bf = cv2.BFMatcher(cv2.NORM_HAMMING)
#
# # match descriptors
# matches: list[DMatch] = bf.match(img_descriptors, template_descriptors)
#
# # sort them in order of their distance
# matches = sorted(matches, key=lambda x: x.distance)
#
# # draw first "percentage" matches. percentage is set to 25
# good = matches[:int(len(matches) * (percentage / 100))]
# img_match = cv2.drawMatches(img, img_key_points, template, template_key_points, good[:50], None,
#                             flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

res = np.empty((max(template.shape[0], img.shape[0]), template.shape[1] + img.shape[1], 3), dtype=np.uint8)
img_match = cv2.drawMatches(template, inliers1, img, inliers2, good_matches, res)
show(img_match)

In [ ]:
# we take all the points in our good matches list and we use them
# to align the corners of our input image
src_points = np.float32([inliers2[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_points = np.float32([inliers1[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

homography, _ = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)

In [ ]:
height, width = template.shape
img_scan = cv2.warpPerspective(img, homography, (width, height))

save('1.img_scan', img_scan)
show(img_scan)

In [ ]:
img_show = img_scan.copy()
img_mask = np.zeros_like(img_show)

In [ ]:
for x, r in enumerate(roi):
    cv2.rectangle(img_mask, (r[0][0], r[0][1]), (r[1][0], r[1][1]), (0, 255, 0), 2)
    img_show = cv2.addWeighted(img_show, 0.99, img_mask, 0.1, 0)
    img_crop = img_scan[r[0][1]:r[1][1], r[0][0]:r[1][0]]
    # img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(img_crop, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # apply a distance transform which calculates the distance to the
    # closest zero pixel for each pixel in the input image
    dist = cv2.distanceTransform(thresh, cv2.DIST_L2, 0)
    # normalize the distance transform such that the distances lie in
    # the range [0, 1] and then convert the distance transform back to
    # an unsigned 8-bit integer in the range [0, 255]
    dist = cv2.normalize(dist, dist, 0, 1, cv2.NORM_MINMAX)
    dist = (dist * 255).astype("uint8")
    # threshold the distance transform using Otsu's method
    dist = cv2.threshold(dist, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 1))
    opening = cv2.morphologyEx(dist, cv2.MORPH_OPEN, kernel)

    # find contours in the opening image, then initialize the list of
    # contours which belong to actual characters that we will be OCR'ing
    cnts = cv2.findContours(dist.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    chars = []
    for c in cnts:
        # compute the bounding box of the contour
        # compute the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)
        # check if contour is at least 35px wide and 100px tall, and if
        # so, consider the contour a digit
        if w >= 10 and h >= 15:
            chars.append(c)

    # compute the convex hull of the characters
    chars = np.vstack([chars[i] for i in range(0, len(chars))])
    hull = cv2.convexHull(chars)
    # allocate memory for the convex hull mask, draw the convex hull on
    # the image, and then enlarge it via a dilation
    mask = np.zeros(img_crop.shape[:2], dtype="uint8")
    cv2.drawContours(mask, [hull], -1, 255, -1)
    mask = cv2.dilate(mask, None, iterations=2)
    cv2.imshow("Mask", mask)
    # take the bitwise of the opening image and the mask to reveal *just*
    # the characters in the image
    final = cv2.bitwise_and(opening, opening, mask=mask)

    show(final)
    save(r[2], final)
    config = '--psm 8'
    if len(r) >= 4:
        config += f' -c tessedit_char_whitelist={r[3]}'

    print(config)
    print(f"{r[2]} : {pytesseract.image_to_string(final, lang='ron', config=config)}")

In [ ]:
blur = cv2.GaussianBlur(img, (0, 0), sigmaX=33, sigmaY=33)
save('2.blured', blur)
show(cv2.bitwise_not(img))

In [ ]:
img = cv2.divide(img, blur, scale=255)
save('3.divided', img)
show(cv2.bitwise_not(img))

In [ ]:
img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
save('4.thresholded', img)
show(cv2.bitwise_not(img))

In [ ]:
lang = "ron"
config = "--psm 11 --oem 1"
text = pytesseract.image_to_string(img, lang=lang, config=config)
print(text)